In [2]:
print("ok")

ok


In [3]:
%pwd

'/workspaces/Medical-Chatbot-Gen-AI/research'

In [24]:
import os
os.chdir("/workspaces/Medical-Chatbot-Gen-AI")
%pwd

'/workspaces/Medical-Chatbot-Gen-AI'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [6]:
def load_pdf_file(data):
    loader= DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [7]:
extracted_data=load_pdf_file(data='Data/')

In [8]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(Chunk_size=100,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [29]:
from langchain.embeddings import HuggingFaceEmbeddings
def download_huggingface_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

# Correct function call
embeddings = download_huggingface_embeddings()

In [30]:
query_result=embeddings.embed_query("Hello World")
print("length", len(query_result))

length 384


In [37]:
from dotenv import load_dotenv
load_dotenv()

True

In [38]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [47]:
pip install pinecone-client

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [48]:
from pinecone import Pinecone
import os
pc = Pinecone(api_key=PINECONE_API_KEY)

index = pc.Index("medibot")



MaxRetryError: HTTPSConnectionPool(host='api.pinecone.io', port=443): Max retries exceeded with url: /indexes/medibot (Caused by SSLError(FileNotFoundError(2, 'No such file or directory')))